In [7]:
import pyspark
from pyspark.sql import SparkSession

In [8]:
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

In [9]:
!wget https://nyc-tlc.s3.amazonaws.com/trip+data/fhvhv_tripdata_2021-01.csv

--2022-02-24 14:04:55--  https://nyc-tlc.s3.amazonaws.com/trip+data/fhvhv_tripdata_2021-01.csv
Resolving nyc-tlc.s3.amazonaws.com (nyc-tlc.s3.amazonaws.com)... 54.231.204.81
Connecting to nyc-tlc.s3.amazonaws.com (nyc-tlc.s3.amazonaws.com)|54.231.204.81|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 752335705 (717M) [text/csv]
Saving to: ‘fhvhv_tripdata_2021-01.csv.1’

fhvhv_tripdata_2021 100%[===================>] 717,48M  10,1MB/s    in 79s     

2022-02-24 14:06:15 (9,06 MB/s) - ‘fhvhv_tripdata_2021-01.csv.1’ saved [752335705/752335705]



In [13]:
!wc -l fhvhv_tripdata_2021-01.csv

 11908469 fhvhv_tripdata_2021-01.csv


In [14]:
df = spark.read \
    .option("header", "true") \
    .csv('fhvhv_tripdata_2021-01.csv')

In [26]:
df.head(5)
# Unlike pandas, Spark does evaluate the types automatically 
# therefore in below we are seeing dates, etc all set as string (Hmm maybe in old versions ;D)

[Row(hvfhs_license_num='HV0003', dispatching_base_num='B02866', pickup_datetime=datetime.datetime(2021, 1, 3, 8, 53, 52), dropoff_datetime=datetime.datetime(2021, 1, 3, 8, 59, 41), PULocationID=37, DOLocationID=36, SR_Flag=None),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02835', pickup_datetime=datetime.datetime(2021, 1, 3, 7, 12, 6), dropoff_datetime=datetime.datetime(2021, 1, 3, 7, 17, 5), PULocationID=80, DOLocationID=37, SR_Flag=None),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02617', pickup_datetime=datetime.datetime(2021, 1, 2, 13, 6, 4), dropoff_datetime=datetime.datetime(2021, 1, 2, 13, 30, 17), PULocationID=181, DOLocationID=249, SR_Flag=None),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02875', pickup_datetime=datetime.datetime(2021, 1, 1, 3, 35, 52), dropoff_datetime=datetime.datetime(2021, 1, 1, 3, 46, 17), PULocationID=213, DOLocationID=32, SR_Flag=None),
 Row(hvfhs_license_num='HV0003', dispatching_base_num='B02872', pickup_datetime

In [27]:
df.schema

StructType(List(StructField(hvfhs_license_num,StringType,true),StructField(dispatching_base_num,StringType,true),StructField(pickup_datetime,TimestampType,true),StructField(dropoff_datetime,TimestampType,true),StructField(PULocationID,IntegerType,true),StructField(DOLocationID,IntegerType,true),StructField(SR_Flag,StringType,true)))

In [28]:
# Keep 1001 rowa in the file and save/create in head.csv
!head -n 1001 fhvhv_tripdata_2021-01.csv > head.csv

In [29]:
import pandas as pd

In [31]:
df_pandas = pd.read_csv('head.csv')

In [32]:
df_pandas.dtypes

hvfhs_license_num        object
dispatching_base_num     object
pickup_datetime          object
dropoff_datetime         object
PULocationID              int64
DOLocationID              int64
SR_Flag                 float64
dtype: object

In [33]:
spark.createDataFrame(df_pandas).schema
# Int takes 4bytes, Long takes 8 bytes

StructType(List(StructField(hvfhs_license_num,StringType,true),StructField(dispatching_base_num,StringType,true),StructField(pickup_datetime,StringType,true),StructField(dropoff_datetime,StringType,true),StructField(PULocationID,LongType,true),StructField(DOLocationID,LongType,true),StructField(SR_Flag,DoubleType,true)))

Integer - 4 bytes
Long - 8 bytes

In [34]:
from pyspark.sql import types

In [36]:
schema = types.StructType([
    types.StructField('hvfhs_license_num', types.StringType(), True),
    types.StructField('dispatching_base_num', types.StringType(), True),
    types.StructField('pickup_datetime', types.TimestampType(), True),
    types.StructField('dropoff_datetime', types.TimestampType(), True),
    types.StructField('PULocationID', types.IntegerType(), True),
    types.StructField('DOLocationID', types.IntegerType(), True),
    types.StructField('SR_Flag', types.StringType(), True)
])

In [37]:
df = spark.read \
    .option("header", "true") \
    .schema(schema) \
    .csv('fhvhv_tripdata_2021-01.csv')

In [38]:
# Idea of repartitioning is to distribute computation in different Spark processors/executors
df = df.repartition(24) # mode="overwrite"

In [40]:
df.write.parquet('fhvhv/2021/01/')

22/02/24 14:42:32 WARN MemoryManager: Total allocation exceeds 95.00% (934,884,135 bytes) of heap memory
Scaling row group sizes to 99.51% for 7 writers
22/02/24 14:42:32 WARN MemoryManager: Total allocation exceeds 95.00% (934,884,135 bytes) of heap memory
Scaling row group sizes to 87.07% for 8 writers
22/02/24 14:42:32 WARN MemoryManager: Total allocation exceeds 95.00% (934,884,135 bytes) of heap memory
Scaling row group sizes to 77.39% for 9 writers
22/02/24 14:42:32 WARN MemoryManager: Total allocation exceeds 95.00% (934,884,135 bytes) of heap memory
Scaling row group sizes to 69.65% for 10 writers
22/02/24 14:42:32 WARN MemoryManager: Total allocation exceeds 95.00% (934,884,135 bytes) of heap memory
Scaling row group sizes to 63.32% for 11 writers
22/02/24 14:42:32 WARN MemoryManager: Total allocation exceeds 95.00% (934,884,135 bytes) of heap memory
Scaling row group sizes to 58.05% for 12 writers
22/02/24 14:42:34 WARN MemoryManager: Total allocation exceeds 95.00% (934,884,

In [52]:
df = spark.read.parquet('fhvhv/2021/01/')

In [53]:
df.printSchema()

root
 |-- hvfhs_license_num: string (nullable = true)
 |-- dispatching_base_num: string (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- SR_Flag: string (nullable = true)



SELECT * FROM df WHERE hvfhs_license_num =  HV0003

In [43]:
from pyspark.sql import functions as F

In [54]:
df.show()

+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|hvfhs_license_num|dispatching_base_num|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|SR_Flag|
+-----------------+--------------------+-------------------+-------------------+------------+------------+-------+
|           HV0005|              B02510|2021-01-03 10:26:53|2021-01-03 10:56:53|         179|          48|   null|
|           HV0005|              B02510|2021-01-01 17:45:58|2021-01-01 18:11:41|         228|         114|   null|
|           HV0005|              B02510|2021-01-01 14:17:55|2021-01-01 14:23:24|          26|          26|   null|
|           HV0003|              B02864|2021-01-01 20:09:27|2021-01-01 20:24:01|         159|          60|   null|
|           HV0003|              B02875|2021-01-03 12:18:07|2021-01-03 12:25:59|         226|         145|   null|
|           HV0005|              B02510|2021-01-03 05:30:44|2021-01-03 05:34:48|

In [45]:
def crazy_stuff(base_num):
    num = int(base_num[1:])
    if num % 7 == 0:
        return f's/{num:03x}'
    elif num % 3 == 0:
        return f'a/{num:03x}'
    else:
        return f'e/{num:03x}'

In [46]:
crazy_stuff('B02884')

's/b44'

In [47]:
# register the python function as udf
crazy_stuff_udf = F.udf(crazy_stuff, returnType=types.StringType())

In [48]:
df \
    .withColumn('pickup_date', F.to_date(df.pickup_datetime)) \
    .withColumn('dropoff_date', F.to_date(df.dropoff_datetime)) \
    .withColumn('base_id', crazy_stuff_udf(df.dispatching_base_num)) \
    .select('base_id', 'pickup_date', 'dropoff_date', 'PULocationID', 'DOLocationID') \
    .show()

+-------+-----------+------------+------------+------------+
|base_id|pickup_date|dropoff_date|PULocationID|DOLocationID|
+-------+-----------+------------+------------+------------+
|  e/9ce| 2021-01-03|  2021-01-03|         179|          48|
|  e/9ce| 2021-01-01|  2021-01-01|         228|         114|
|  e/9ce| 2021-01-01|  2021-01-01|          26|          26|
|  e/b30| 2021-01-01|  2021-01-01|         159|          60|
|  e/b3b| 2021-01-03|  2021-01-03|         226|         145|
|  e/9ce| 2021-01-03|  2021-01-03|         244|         244|
|  a/b37| 2021-01-03|  2021-01-03|         234|         224|
|  e/9ce| 2021-01-02|  2021-01-02|          47|         159|
|  e/b42| 2021-01-01|  2021-01-01|          82|         157|
|  a/a7a| 2021-01-01|  2021-01-01|         223|         198|
|  e/b38| 2021-01-01|  2021-01-01|         246|          48|
|  e/9ce| 2021-01-01|  2021-01-01|          65|          25|
|  e/b3e| 2021-01-01|  2021-01-01|          13|         230|
|  e/9ce| 2021-01-03|  2

In [55]:
df.select('pickup_datetime', 'dropoff_datetime', 'PULocationID', 'DOLocationID') \
  .filter(df.hvfhs_license_num == 'HV0003')


DataFrame[pickup_datetime: timestamp, dropoff_datetime: timestamp, PULocationID: int, DOLocationID: int]

In [56]:
!head -n 10 head.csv